NTP pools
- /etc/systemd/timesyncd.conf
- sudo systemctl restart systemd-timesyncd


Cron Jobs
- user vs system wide jobs
    - sudo su john
    - crontab -e

Networking
- Redirection Rules
    - sudo iptables -t nat -A PREROUTING -p tcp -s 10.5.5.0/24 --dport 81 -j DNAT --to-destination 192.168.5.2:80
- MAsqureading 
    - sudo iptables -t nat -A POSTROUTING -s 10.5.5.0/24 -j MASQUERADE
- iptables rules
    - persistance
         - sudo apt install iptables-persistent
- Create network Bridge
- Enable/Disable DHCP

Add files to ACL (https://www.redhat.com/en/blog/linux-access-control-lists)
- getfacl
- Set read and write permissions for janet on /opt/aclfile:
    - sudo setfacl -m u:janet:rw /opt/aclfile


Security Limits
```bash
/etc/security/limits.conf 
```

```bash
janet hard nproc 100
@mail soft fsize 8192
```

SElinux
- use sestatus to confirm SElinux mode
- use chcon to change SElinux labels

Check the current SELinux status on node01 and store it in file /opt/selinuxmode.txt:
- getenforce | sudo tee /opt/selinuxmode.txt

Correct SELinux Type Label for /usr/bin/less:
- sudo restorecon -v /usr/bin/less

To find out which files are not a certificate, run the below command, and if you get output as Unable to load certificate, then it is a key.
- openssl x509 -in file* -noout -text

To find out the certificate that is using the 2048 bit, run the below command:
- openssl x509 -in file* -noout -text | grep "Public-Key"

## LVM
Add /dev/vdc to volume1:
- sudo vgextend volume1 /dev/vdc

Resize the Logical Volume lv1 to 2GB:
- sudo lvresize --size 2G /dev/volume1/lv1

Create a virtual machine with the following parameters on caleston-lp10:

For the operating system information parameter use the ubuntu22.04.
Name the virtual machine mockexam2.
Assign 1024 MB of RAM to this machine.
Assign it one virtual CPU.
Import the disk image /var/lib/libvirt/images/ubuntu.img to this virtual machine.
At the end of your command, you can add the parameter --noautoconsole to avoid waiting for this virtual machine to boot up, and not get stuck in the virtual console after it initializes.
After you create this virtual machine, run a separate command to make mockexam2 automatically start up every time the system boots up.

Run the below command to launch vm with ubuntu22.04 image:
```bash
virt-install \
--name mockexam2 \
--ram 1024 \
--vcpus=1 \
--os-variant=ubuntu22.04 \
--import \
--disk path=/var/lib/libvirt/images/ubuntu.img \
--noautoconsole
```
Configure the virtual machine to start automatically on boot:
```bash
virsh autostart mockexam2
```

Create a network bridge between these network interfaces: [ "eth1", and "eth2"] on node01. Call the bridge bridge1. Turn DHCP off for IPv4 for both interfaces. However, for the bridge itself, turn DHCP on for IPv4.

To create a bridge network with eth1 and eth2, create a file /etc/netplan/99-bridge.yaml and edit the file as below:
```bash
network:
  version: 2
  renderer: networkd
  ethernets:
    eth1:
      dhcp4: no
    eth2:
      dhcp4: no
  bridges:
    bridge1:
      dhcp4: yes
      interfaces:
        - eth1
        - eth2
```
Apply the new network configuration using the netplan apply command

The node01 was configured to use LDAP entries from a certain server. However, some configuration options are wrong. Edit the correct configuration files and fix the following mistakes:

Our name service local daemon is configured to look for an LDAP server at the wrong IP address (currently 10.9.9.8). Fix this and configure the correct IP to find it run ip -4 addr show eth0 ignoring /24 . Make sure your changes become active after you edit the configuration file.

Our system is currently configured to get group data and password data from the LDAP server but not user data. Configure it to get user data as well.

Credentials to access node01:
Name: bob
Password: caleston123

Switch back to caleston-lp10 once the task is done

Edit /etc/nslcd.conf to change the ldap server:
```bash
# The user and group nslcd should run as.
uid nslcd
gid nslcd

# The location at which the LDAP server(s) should be reachable.
uri ldap://192.168.121.167/ # updated it with the output of ``ip -4 addr show eth0`

# The search base that will be used for all queries.
base dc=example,dc=org

# The LDAP protocol version to use.
#ldap_version 3

# The DN to bind with for normal lookups.
#binddn cn=annonymous,dc=example,dc=net
#bindpw secret
```

Edit /etc/nsswitch.conf to get user data:
```bash
# /etc/nsswitch.conf
#
# Example configuration of GNU Name Service Switch functionality.
# If you have the `glibc-doc-reference' and `info' packages installed, try:
# `info libc "Name Service Switch"' for information about this file.

passwd:         files systemd ldap #updated
group:          files systemd ldap
shadow:         files ldap
gshadow:        files
```

Restart the nslcd service:
```bash
sudo systemctl restart nslcd
```